In [19]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

In [20]:
import os
import glob
import datetime as dt
import numpy as np
import torch

import neural_lam.constants as constants

In [21]:
# load time step data
# sample_dir_path = os.path.join(BASE_PATH, "data/era5_uk/samples/train")
sample_dir_path = "data/era5_uk/samples/train"

sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
init_states = torch.tensor(np.load(sample_files[0]), dtype=torch.float32).unsqueeze(0)
sample_files = [os.path.basename(f) for f in sample_files]
sample_times = [dt.datetime.strptime(f, '%Y%m%d%H%M%S.npy') for f in sample_files]

In [18]:
sample_files

['20220101000000.npy',
 '20220101060000.npy',
 '20220101120000.npy',
 '20220101180000.npy',
 '20220102000000.npy',
 '20220102060000.npy',
 '20220102120000.npy',
 '20220102180000.npy',
 '20220103000000.npy',
 '20220103060000.npy',
 '20220103120000.npy',
 '20220103180000.npy',
 '20220104000000.npy',
 '20220104060000.npy',
 '20220104120000.npy',
 '20220104180000.npy',
 '20220105000000.npy',
 '20220105060000.npy',
 '20220105120000.npy',
 '20220105180000.npy',
 '20220106000000.npy',
 '20220106060000.npy',
 '20220106120000.npy',
 '20220106180000.npy',
 '20220107000000.npy',
 '20220107060000.npy',
 '20220107120000.npy',
 '20220107180000.npy',
 '20220108000000.npy',
 '20220108060000.npy',
 '20220108120000.npy',
 '20220108180000.npy',
 '20220109000000.npy',
 '20220109060000.npy',
 '20220109120000.npy',
 '20220109180000.npy',
 '20220110000000.npy',
 '20220110060000.npy',
 '20220110120000.npy',
 '20220110180000.npy',
 '20220111000000.npy',
 '20220111060000.npy',
 '20220111120000.npy',
 '202201111

In [ ]:
i = 2
sample_length = 6 + 2

# times = sample_times[i:i+sample_length]

hour_inc = torch.arange(sample_length) * 6 # (N_t,)

init_hour = sample_times[i].hour
hour_of_day = init_hour + hour_inc

start_of_year = dt.datetime(sample_times[i].year, 1, 1)
init_seconds_into_year = (sample_times[i] - start_of_year).total_seconds()
seconds_into_year = init_seconds_into_year + hour_inc * 3600

hour_angle = (hour_of_day / 24) * 2 * torch.pi 
year_angle = (seconds_into_year / constants.SECONDS_IN_YEAR) * 2 * torch.pi

In [ ]:
hour_angle

tensor([ 3.1416,  4.7124,  6.2832,  7.8540,  9.4248, 10.9956, 12.5664, 14.1372])

In [ ]:
year_angle

tensor([0.0086, 0.0129, 0.0172, 0.0215, 0.0258, 0.0301, 0.0344, 0.0387])

In [ ]:
datetime_forcing = torch.stack(
    (
        torch.sin(hour_angle),
        torch.cos(hour_angle),
        torch.sin(year_angle),
        torch.cos(year_angle),
    ),
    dim=1,
)  # (N_t, 4)
datetime_forcing = (datetime_forcing + 1) / 2 # Normalize to [0,1]
print(datetime_forcing.shape)
datetime_forcing = datetime_forcing.unsqueeze(1).expand(
    -1, init_states.shape[1], -1
)  # (N_t+1, N_grid, 4)
print(datetime_forcing.shape)

forcing = torch.cat(
    (
        datetime_forcing[:-2],
        datetime_forcing[1:-1],
        datetime_forcing[2:],
    ),
    dim=2,
)

print(forcing.shape)

torch.Size([8, 4])
torch.Size([8, 3705, 4])
torch.Size([6, 3705, 12])


tensor([[-8.7423e-08, -1.0000e+00,  8.6070e-03,  9.9996e-01],
        [-1.0000e+00,  1.1925e-08,  1.2910e-02,  9.9992e-01],
        [ 1.7485e-07,  1.0000e+00,  1.7213e-02,  9.9985e-01],
        [ 1.0000e+00, -3.3777e-07,  2.1516e-02,  9.9977e-01],
        [-2.3850e-08, -1.0000e+00,  2.5818e-02,  9.9967e-01],
        [-1.0000e+00,  6.6361e-07,  3.0120e-02,  9.9955e-01],
        [ 3.4969e-07,  1.0000e+00,  3.4422e-02,  9.9941e-01],
        [ 1.0000e+00, -3.5775e-08,  3.8722e-02,  9.9925e-01]])

In [ ]:
hour_of_day

tensor([12, 18, 24, 30, 36, 42, 48, 54])

In [ ]:
_hour_of_day

tensor([12, 18,  0,  6, 12, 18,  0,  6])

In [ ]:
hour_of_day

tensor([12, 18,  0,  6, 12, 18,  0,  6])

In [ ]:
dt_obj.hour

int